In [1]:
import sys
sys.path.append('C:\Program Files\Anaconda3\Lib\site-packages')

In [2]:
import gensim
import json
import re
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim 

morph = MorphAnalyzer()

C:\Program Files\Anaconda3\Lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Slow version of gensim.models.doc2vec is being used


In [3]:
from tqdm import tqdm_notebook

## Домашнее задание

Основаная задача - **построить хорошую тематическую модель с интерпретируемыми топиками**.

1) сделайте нормализацию (если pymorphy2 работает долго используйте mystem или попробуйте установить быструю версию - `pip install pymorphy2[fast]`, можно использовать какой-то другой токенизатор); 

2) добавьте нграммы (в тетрадке есть закомменченая ячейка с Phrases); 

3) сделайте хороший словарь (отфильтруйте слишком частотные и редкие слова, попробуйте удалить стоп-слова); 

4) постройте несколько LDA моделей (переберите количество тем, можете поменять eta, alpha, passes), если получаются плохие темы, поработайте дополнительно над предобработкой и словарем; 

5) для самой хорошей модели в отдельной ячейке напечатайте 3 хороших (на ваш вкус) темы;

6) между словарем и обучением модели добавьте tfidf (`gensim.models.TfidfModel(corpus, id2word=dictionary); corpus = tfidf[corpus]`);

7) заново обучите LDA c теми же параметрами (параметрами самой лучшей модели, заново перебирать не нужно);

8) в отдельной ячейке опишите как изменилась модель (приведите несколько тем, которые стали лучше или хуже, или которых раньше вообще не было; можно привести значения перплексии и когерентности для обеих моделей)

9) проделайте такие же действия для NMF (образец в конце тетрадки), для построения словаря воспользуйтесь возможностями Count или Tfidf Vectorizer (попробуйте другие значение max_features, min_df, max_df, сделайте нграмы через ngram_range, если хватает памяти), попробуйте такие же количества тем

10) в отдельной ячейки напечатайте таблицу с темами лучшей NMF модели, сравните их с теми, что получились в LDA.

Сохраните тетрадку с экспериментами и положите её на гитхаб, ссылку на неё укажите в форме.

**Оцениваться будут главным образом пункты 5, 8 и 10. (2, 3, 2 баллов соответственно). Чтобы заработать остальные 3 балла, нужно хотя бы немного изменить мой код на промежуточных этапах (добавить что-то, указать другие параметры и т.д). **

## Данные

#### Токенизация, нормализация, удаление стоп-слов для русского

In [4]:
def remove_tags(text):
    return re.sub(r'<[^>]+>', '', text)

def clean(text, stopwords):
    words = re.findall(r'[a-zа-яё]+', text.lower())
    clean = [morph.parse(word)[0].normal_form for word in words if word not in stopwords]
    return clean

In [5]:
from stop_words import get_stop_words
stopwords = get_stop_words('russian')

Возьмем 4 тыс статьи с Хабра. Это мало для хорошей тематической модели, но иначе у нас просто ничего не обучится за семинар.

In [6]:
texts = open('habr_texts.txt', encoding = 'utf-8').read().splitlines()

In [7]:
texts[0][:900]

'MassTransit это open source библиотека, разработанная на языке C# для .NET платформы, упрощающая работу с шиной данных, которая используется при построении распределенных приложений и реализации SOA (service oriented architecture).\tВ качестве message broker могут выступать RabbitMq, Azure Service Bus или In-Memory менеджер (в случае с In-Memory область видимости ограничивается процессом, в котором проинициализирован экземпляр).Содержание:Команды и событияКоманды\tСобытия\t\tКонтракты сообщений\tРоутингExchange\tФормат сообщения\t\tКонсьюмеры (Consumer)\tКонфигурация контейнера DI\tНаблюдатели (Observer)\tНовое в MassTransit 3.0\tЗаключение\tОпрос: А какую .NET библиотеку используете вы?\tКоманды и события\tВ библиотеке заложено 2 основных типа сообщений: команды и события.Команды\tСигнализируют о необходимости выполнить некое действие. Для наиболее содержательного наименования команды желательно исполь'

В текстах есть тэги. Потрем их. Ещё токенизируем самым простым способом и нормализуем Pymorphy.

In [9]:
texts = [clean(text, stopwords) for text in tqdm_notebook(texts)]

#### Удаление дополнительных стоп-слов

In [39]:
texts = [[token for token in text if len(token) > 2] for text in tqdm_notebook(texts)]

In [48]:
stopwords_eng = get_stop_words('english')

In [50]:
texts = [[token for token in text if token not in stopwords_eng] for text in tqdm_notebook(texts)]

In [14]:
texts[0][:10]

['masstransit',
 'open',
 'source',
 'библиотека',
 'разработать',
 'язык',
 'net',
 'платформа',
 'упрощать',
 'работа']

In [15]:
text_list = [' '.join(text) for text in texts]

In [37]:
text_list[0][:900]

'masstransit open source библиотека разработать язык net платформа упрощать работа шина дать использоваться построение распределенный приложение реализация soa service oriented architecture качество message broker выступать rabbitmq azure service bus memory менеджер случай memory область видимость ограничиваться процесс который проинициализировать экземпляр содержание команда событиякоманда событие контракт сообщение роутингexchange формат сообщение консьюмер consumer конфигурация контейнер наблюдатель observer новое masstransit заключение опрос какой net библиотека использовать команда событие библиотека заложить основный тип сообщение команда событие команда сигнализировать необходимость выполнить некий действие содержательный наименование команда желательно использовать структура глагол существительное estimateconnection sendsms notifycustomerorderprocessed работа команда осуществлятьс'

In [54]:
with open('habr_text_clean.txt', 'w', encoding = 'utf-8')as f:
    f.write('\n'.join(text_list))

In [8]:
texts = open('habr_text_clean.txt', encoding = 'utf-8').read().splitlines()

In [13]:
texts = [text.split() for text in texts]

#### N-граммы

In [39]:
# для нграммов
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.4) # threshold можно подбирать
p = gensim.models.phrases.Phraser(ph)


C:\Program Files\Anaconda3\Lib\site-packages\gensim\models\phrases.py:275: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if all(parameter in getargspec(scoring)[0] for parameter in scoring_parameters):
C:\Program Files\Anaconda3\Lib\site-packages\gensim\models\phrases.py:275: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if all(parameter in getargspec(scoring)[0] for parameter in scoring_parameters):
C:\Program Files\Anaconda3\Lib\site-packages\gensim\models\phrases.py:275: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if all(parameter in getargspec(scoring)[0] for parameter in scoring_parameters):
C:\Program Files\Anaconda3\Lib\site-packages\gensim\models\phrases.py:275: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if all(parameter in getargspec(scoring)[0] for parameter in scoring_parameters):
C:\Program Files

In [40]:
len(ph.vocab)

2166392

In [41]:
ngrammed_texts = [p[text] for text in texts]

In [42]:
print(ngrammed_texts[0][:100])

['masstransit', 'open_source', 'библиотека', 'разработать', 'язык', 'net', 'платформа', 'упрощать', 'работа', 'шина', 'дать', 'использоваться', 'построение', 'распределенный', 'приложение', 'реализация', 'soa_service', 'oriented_architecture', 'качество', 'message', 'broker_выступать', 'rabbitmq', 'azure', 'service', 'bus', 'memory', 'менеджер', 'случай', 'memory', 'область_видимость', 'ограничиваться', 'процесс', 'который', 'проинициализировать_экземпляр', 'содержание', 'команда_событиякоманда', 'событие', 'контракт', 'сообщение_роутингexchange', 'формат', 'сообщение', 'консьюмер_consumer', 'конфигурация', 'контейнер', 'наблюдатель_observer', 'новое', 'masstransit_заключение', 'опрос', 'какой', 'net', 'библиотека', 'использовать', 'команда', 'событие', 'библиотека', 'заложить', 'основный', 'тип', 'сообщение', 'команда', 'событие', 'команда', 'сигнализировать', 'необходимость', 'выполнить', 'некий', 'действие', 'содержательный_наименование', 'команда', 'желательно', 'использовать', 'ст

### Тематическое моделирование в gensim

Для моделей нужно сделать словарь.

In [101]:
dictionary_unigram = gensim.corpora.Dictionary(texts)

In [102]:
len(dictionary_unigram)

124785

In [43]:
# dictinary = gensim.corpora.Dictionary(texts)
dictionary = gensim.corpora.Dictionary(ngrammed_texts)

In [44]:
len(dictionary)

357705

In [45]:
dictionary.filter_extremes(no_above=0.3, no_below=30)
dictionary.compactify()

In [46]:
len(dictionary)

5636

In [63]:
print(dictionary)

Dictionary(5636 unique tokens: ['числовой', 'модула', 'фактор', 'наименьший', 'статичный']...)


Преобразуем наши тексты в мешки слов. 

In [47]:
corpus = [dictionary.doc2bow(text) for text in ngrammed_texts]
# если текстов много, то тут может быть генератор

In [ ]:
num_topics = [20, 30, 50]
lda_models = []

for num in tqdm_notebook(num_topics):
    lda = gensim.models.LdaModel(corpus, 50, id2word=dictionary, passes=5)
    perp = lda.log_perplexity(corpus[:2000], total_docs=100)
    coherence_model_lda = gensim.models.CoherenceModel(model=lda, 
                                                   texts=ngrammed_texts, 
                                                   dictionary=dictionary, coherence='c_v')
    coher = coherence_model_lda.get_coherence()
    print('Topics: {}'.format(num))
    print('Perplexity: {}'.format(perp))
    print('Coherence: {}'.format(coher))
    lda_models.append(lda)

Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\Program Files\Anaconda3\envs\mllecture\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Program Files\Anaconda3\Lib\site-packages\tqdm\_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "C:\Program Files\Anaconda3\envs\mllecture\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



Topics: 20
Perplexity: -14.572874560746872
Coherence: 0.4703812428838745
Topics: 30
Perplexity: -14.758817349420916
Coherence: 0.47454308776086584
Topics: 50
Perplexity: -14.772974197046425
Coherence: 0.48281713702374346


In [54]:
%%time
lda = gensim.models.LdaModel(corpus, 100, id2word=dictionary, passes=5)
perp = lda.log_perplexity(corpus[:2000], total_docs=100)
coherence_model_lda = gensim.models.CoherenceModel(model=lda, 
                                               texts=ngrammed_texts, 
                                               dictionary=dictionary, coherence='c_v')
coher = coherence_model_lda.get_coherence()
print('Topics: {}'.format(num))
print('Perplexity: {}'.format(perp))
print('Coherence: {}'.format(coher))

Topics: 100
Perplexity: -17.030831860117406
Coherence: 0.43778241186315325
Wall time: 13min 15s


In [66]:
%%time
lda = gensim.models.LdaModel(corpus, 50, id2word=dictionary, passes=5)

Wall time: 3min 15s


Посмотрим на топики.

In [87]:
lda.print_topics(num_topics=50)

[(0,
  '0.034*"боль" + 0.016*"изменение" + 0.012*"windows" + 0.012*"компьютер" + 0.011*"процессор" + 0.009*"microsoft" + 0.009*"происходить" + 0.009*"ноутбук" + 0.009*"ситуация" + 0.008*"экран"'),
 (1,
  '0.038*"устройство" + 0.019*"раздел" + 0.017*"windows" + 0.014*"dev" + 0.011*"master" + 0.010*"system" + 0.010*"пароль" + 0.009*"загрузка" + 0.008*"обновление" + 0.008*"файл"'),
 (2,
  '0.022*"язык" + 0.020*"программа" + 0.016*"технология" + 0.016*"компьютер" + 0.013*"программист" + 0.013*"программирование" + 0.011*"метод" + 0.009*"математика" + 0.009*"материал" + 0.008*"печать"'),
 (3,
  '0.118*"браузер" + 0.051*"страница" + 0.030*"chrome" + 0.023*"вкладка" + 0.019*"таймер" + 0.014*"секунда" + 0.013*"фоновый" + 0.012*"веб" + 0.012*"загрузка" + 0.012*"microsoft"'),
 (4,
  '0.027*"мозг" + 0.021*"машина" + 0.014*"сигнал" + 0.012*"робот" + 0.012*"лицо" + 0.011*"учёный" + 0.009*"область" + 0.008*"прибор" + 0.007*"определить" + 0.007*"движение"'),
 (5,
  '0.123*"открытый" + 0.059*"публикаци

### Наиболее удачная модель получилась для 50 тем. 
#### 3 хорошие темы:

In [81]:
for topic_id, topic in lda.show_topics(num_topics=50, formatted=False):
    if topic_id in [43, 24, 33]:
        print(topic)

[('вакансия', 0.020873231100548874), ('сотрудник', 0.011671128412801865), ('деньга', 0.010646055995910479), ('опыт', 0.0091200151439239704), ('документ', 0.0084403813258911737), ('офис', 0.0081656577607280428), ('продукт', 0.0068450624229650491), ('месяц', 0.0061280930467886548), ('крупный', 0.0055194629237798436), ('навык', 0.0054954172300921127)]
[('атака', 0.49716742427732868), ('злоумышленник', 0.15313274948355565), ('жертва', 0.084886116210557305), ('хакер', 0.063414706962569109), ('ddos_атака', 0.047165486290493927), ('суметь', 0.0057275881205254642), ('банка', 0.0054555546839932731), ('веб_сайт', 0.0049324879668635078), ('интернет_вещий', 0.0043091749946214831), ('октябрь', 0.0034649329737704545)]
[('бот', 0.022158978925882488), ('контент', 0.014528241124237095), ('перевод', 0.013267550733086888), ('текст', 0.012003804955109946), ('язык', 0.011525450340974717), ('сообщение', 0.01089963002475774), ('поиск', 0.010602058471327929), ('google', 0.008940158926155321), ('twitter', 0.00

Ещё есть штука для визуализации.

In [68]:
pyLDAvis.enable_notebook()

In [69]:
pyLDAvis.gensim.prepare(lda, corpus, dictionary)

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
17     7.132784        1       1  0.173514 -0.040097
29     5.369140        1       2  0.156902 -0.040287
12     4.835360        1       3  0.155015  0.007553
39     4.279243        1       4  0.190323  0.022326
49     3.973851        1       5 -0.161071 -0.172409
9      3.957831        1       6 -0.069405 -0.137124
34     3.687740        1       7  0.015366 -0.104203
8      3.588991        1       8  0.109056  0.002777
31     3.555824        1       9  0.173705  0.027370
13     3.413610        1      10  0.121330 -0.055322
6      3.220675        1      11 -0.048468 -0.078517
14     3.102181        1      12  0.053354 -0.056736
47     3.019913        1      13  0.078948 -0.094085
7      2.824403        1      14 -0.224356 -0.149367
15     2.751103        1      15 -0.153796 -0.147285
27     2.729138        1      16 -0.143895 -0.135995
36     2.727445        1      17  0.088849 -0.032310
24     2.713461        1      18  0.183144  0.023814
35     2.593425        1      19  0.090799 -0.037032
20     2.377517        1      20  0.102757 -0.026062
2      2.166111        1      21  0.121393  0.010729
0      2.029364        1      22  0.096199  0.002508
41     1.848831        1      23  0.136166 -0.041838
25     1.704175        1      24  0.020343 -0.031871
4      1.692202        1      25  0.136147  0.021417
40     1.624719        1      26 -0.124419 -0.059739
43     1.539491        1      27  0.083646 -0.037316
1      1.359920        1      28 -0.094955 -0.061649
45     1.344974        1      29 -0.055458 -0.089879
22     1.219127        1      30 -0.240163 -0.034188
46     1.102597        1      31 -0.051116 -0.038657
16     1.013864        1      32  0.060269  0.098738
19     0.966892        1      33 -0.054683 -0.037344
23     0.953727        1      34  0.081802  0.007334
28     0.854186        1      35 -0.069802 -0.072766
11     0.829168        1      36 -0.087523 -0.027571
48     0.755463        1      37  0.049788 -0.015244
30     0.594839        1      38  0.011169  0.048287
32     0.574539        1      39  0.026930  0.135506
37     0.569424        1      40 -0.211391  0.079089
42     0.480385        1      41 -0.052875  0.123271
3      0.476434        1      42 -0.068138  0.099211
38     0.435674        1      43  0.034490  0.183908
26     0.371181        1      44  0.028949  0.001220
18     0.358394        1      45 -0.155007  0.195849
21     0.347630        1      46 -0.162061  0.124902
44     0.321107        1      47 -0.035285  0.101905
5      0.278076        1      48 -0.100274  0.160893
10     0.236570        1      49 -0.115001  0.118879
33     0.097298        1      50 -0.101211  0.257409, topic_info=     Category         Freq                 Term        Total  loglift  logprob
term                                                                          
1503  Default  6404.000000                 игра  6404.000000  30.0000  30.0000
5407  Default  4313.000000                метод  4313.000000  29.0000  29.0000
4979  Default  4605.000000               объект  4605.000000  28.0000  28.0000
5563  Default  7220.000000                 файл  7220.000000  27.0000  27.0000
2513  Default  4555.000000             значение  4555.000000  26.0000  26.0000
3756  Default  4847.000000           устройство  4847.000000  25.0000  25.0000
5591  Default  4010.000000                 сеть  4010.000000  24.0000  24.0000
5333  Default  3322.000000                 сайт  3322.000000  23.0000  23.0000
64    Default  1509.000000                книга  1509.000000  22.0000  22.0000
5280  Default  1433.000000               кнопка  1433.000000  21.0000  21.0000
1438  Default  2951.000000               return  2951.000000  20.0000  20.0000
1697  Default  1592.000000              браузер  1592.000000  19.0000  19.0000
842   Default  2036.000000                канал  2036.000000  18.0000  18.0000
762   Default  1

Можно посмотреть метрики.

In [48]:
import numpy as np

In [71]:
?lda.log_perplexity

In [72]:
lda.log_perplexity(corpus[:2000], total_docs=100)

-14.634601528516317

In [74]:
#temp_file = datapath("model")
lda.save('lda_model50')

In [88]:
coherence_model_lda = gensim.models.CoherenceModel(model=lda, 
                                                   texts=ngrammed_texts, 
                                                   dictionary=dictionary, coherence='c_v')

In [89]:
coherence_model_lda.get_coherence()

0.46530631812397522

In [76]:
# topics = []
# for topic_id, topic in lda.show_topics(num_topics=50, formatted=False):
#     topic = [word for word, _ in topic]
#     topics.append(topic)

In [78]:
# coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
#                                                    texts=texts, 
#                                                    dictionary=dictionary, coherence='c_v')

In [80]:
# coherence_model_lda.get_coherence()

#### Модель с TFIDF

In [82]:
tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary) 
corpus = tfidf[corpus]

In [84]:
%%time
lda_tfidf = gensim.models.LdaModel(corpus, 50, id2word=dictionary, passes=5)

Wall time: 2min 10s


In [86]:
lda_tfidf.print_topics(num_topics=50)

[(0,
  '0.006*"игра" + 0.002*"игрок" + 0.002*"технология" + 0.002*"модель" + 0.002*"язык" + 0.002*"браузер" + 0.002*"инструмент" + 0.002*"элемент" + 0.002*"канал" + 0.002*"продукт"'),
 (1,
  '0.022*"хостинг_провайдер" + 0.011*"затратный" + 0.000*"хостинг" + 0.000*"vps" + 0.000*"основатель" + 0.000*"аудит" + 0.000*"jira" + 0.000*"инстанс" + 0.000*"российский_федерация" + 0.000*"инвестор"'),
 (2,
  '0.034*"public_function" + 0.031*"cache" + 0.030*"кэш" + 0.029*"disk" + 0.028*"admin" + 0.024*"домен" + 0.018*"storage" + 0.016*"перезагрузка" + 0.016*"скрипт" + 0.015*"auth"'),
 (3,
  '0.000*"ступень" + 0.000*"консультант" + 0.000*"океан" + 0.000*"клиент" + 0.000*"заявка" + 0.000*"переключать" + 0.000*"профиль" + 0.000*"подписчик" + 0.000*"стартовый" + 0.000*"баннер"'),
 (4,
  '0.119*"вакансия" + 0.092*"кампания" + 0.014*"переноситься" + 0.007*"газета" + 0.003*"big_data" + 0.002*"разбиение" + 0.002*"соревнование" + 0.000*"мобильный_приложение" + 0.000*"теряться" + 0.000*"медиа"'),
 (5,
  '0.0

#### Темы с использованием tfidf:
1. Появились хорошие темы про космос и ракетостроение (5 ,46), технику (33).
2. Ухудшилась тема про вакансии(4)
3. В целом, среди тематических слов стало выделяться больше узко специализированных, но нельзя сказать, что это способствовало улучшению тем.

In [90]:
# Хорошие темы с tfidf
for topic_id, topic in lda_tfidf.show_topics(num_topics=50, formatted=False):
    if topic_id in [5, 46, 33]:
        print(topic)

[('вселенная', 0.030973434198393178), ('спутник', 0.025627251541927826), ('земля', 0.023139257683205731), ('планета', 0.021764034974532907), ('орбита', 0.020052366402275479), ('звезда', 0.019988816789823931), ('космический', 0.019872484198144857), ('космос', 0.01970554564769247), ('наса', 0.019247664688637218), ('луна', 0.018856644871184228)]
[('температура', 0.028751411404110423), ('аккумулятор', 0.023660325878241344), ('прибор', 0.023094525383055783), ('печать', 0.023053546335329852), ('датчик', 0.021883196578493139), ('усилитель', 0.019234066873909113), ('клетка', 0.01825340674633898), ('принтер', 0.015279006618447001), ('батарея', 0.013001155589774296), ('кожа', 0.012702985369024958)]
[('двигатель', 0.084181459417592225), ('пуск', 0.029073062146208854), ('ракета', 0.027937429605365407), ('ракета_носитель', 0.027099312900960119), ('союз', 0.022821237873268112), ('атом', 0.019573539301416077), ('военный', 0.0193836400501422), ('металлический', 0.017065257995178641), ('орбита', 0.0164

In [91]:
# Тема с вакансией в модели с tfidf
for topic_id, topic in lda_tfidf.show_topics(num_topics=50, formatted=False):
    if topic_id in [4]:
        print(topic)

[('вакансия', 0.11878431566821304), ('кампания', 0.091962485722332454), ('переноситься', 0.014156730226659888), ('газета', 0.0066254025755466527), ('big_data', 0.0025444111739044537), ('разбиение', 0.0024035244868802421), ('соревнование', 0.0023276314333719669), ('мобильный_приложение', 0.00031642564476031849), ('теряться', 0.00018999512489345835), ('медиа', 0.00016080682025946123)]


In [92]:
# Тема с вакансией в модели без tfidf
for topic_id, topic in lda.show_topics(num_topics=50, formatted=False):
    if topic_id in [24]:
        print(topic)

[('вакансия', 0.020873231100548874), ('сотрудник', 0.011671128412801865), ('деньга', 0.010646055995910479), ('опыт', 0.0091200151439239704), ('документ', 0.0084403813258911737), ('офис', 0.0081656577607280428), ('продукт', 0.0068450624229650491), ('месяц', 0.0061280930467886548), ('крупный', 0.0055194629237798436), ('навык', 0.0054954172300921127)]


In [93]:
lda_tfidf.log_perplexity(corpus[:2000], total_docs=100)

-41.189391565870878

In [94]:
coherence_model_lda = gensim.models.CoherenceModel(model=lda_tfidf, 
                                                   texts=ngrammed_texts, 
                                                   dictionary=dictionary, coherence='c_v')
coherence_model_lda.get_coherence()

0.4405139421176188

#### Метрики для двух моделей:
1. Перплексия значительно ниже у модели без tfidf.
2. Когерентность немного выше у модели без tfidf. <br>
Показатели модели без tfidf лучше, и по анализу тематических слов можно сказать, что темы модели без tfidf получились более интерпретируемыми. Возможно модель с tfidf дала бы более хорошие темы, если отфильтровать словарь по tfidf (оставив наиболее значимые слова).

### Разложение матриц в sklearn

In [17]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

Sklearn принимает на вход строки, поэтому склеим наши списки.

In [18]:
text_list[0][:500]

'masstransit open source библиотека разработать язык net платформа упрощать работа шина дать использоваться построение распределенный приложение реализация soa service oriented architecture качество message broker выступать rabbitmq azure service bus memory менеджер случай memory область видимость ограничиваться процесс который проинициализировать экземпляр содержание команда событиякоманда событие контракт сообщение роутингexchange формат сообщение консьюмер consumer конфигурация контейнер наблю'

Сделаем матрицу слова-документы с помощью TfidfVectorizer

In [19]:
vectorizer = TfidfVectorizer(max_features=50000, min_df=3, max_df=0.3, lowercase=False)
X = vectorizer.fit_transform(text_list)

Разложим её.

In [26]:
def get_nmf_topics(model, n_top_words, ):
    
    #id слов.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(model.n_components):
        
        #топ n слов для темы.
        words_ids = model.components_[i].argsort()[:-n_top_words - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

In [21]:
num_topics = [20, 30, 50]

In [28]:
topicsdf = []
for num in tqdm_notebook(num_topics):
    model = NMF(n_components=num)
    model.fit(X)
    df = get_nmf_topics(model, 10)
    topicsdf.append((model, df))
    

Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Program Files\Anaconda3\envs\mllecture\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Program Files\Anaconda3\Lib\site-packages\tqdm\_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "C:\Program Files\Anaconda3\envs\mllecture\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



In [32]:
topicsdf[0][1].T

,0,1,2,3,4,5,6,7,8,9
Topic # 01,бизнес,сотрудник,продукт,клиент,деньга,рынок,заказчик,crm,менеджер,программист
Topic # 02,файл,php,http,docker,сервер,file,скрипт,sudo,amp,user
Topic # 03,игра,игрок,игровой,играть,персонаж,unity,steam,движок,геймплей,жанр
Topic # 04,язык,обучение,программирование,алгоритм,программа,нейросеть,программист,лекция,модель,сеть
Topic # 05,сайт,страница,реклама,браузер,контент,google,клиент,рекламный,домен,com
Topic # 06,устройство,камера,смартфон,звук,сигнал,датчик,корпус,принтер,аккумулятор,наушник
Topic # 07,public,string,return,void,int,new,class,класс,null,var
Topic # 08,космический,спутник,ракета,орбита,марс,аппарат,земля,луна,полёт,наса
Topic # 09,react,css,javascript,div,angular,props,компонент,dom,веб,html
Topic # 10,атака,безопасность,уязвимость,устройство,злоумышленник,защита,пароль,вредоносный,сеть,доступ


In [33]:
topicsdf[1][1].T

,0,1,2,3,4,5,6,7,8,9
Topic # 01,клиент,продукт,бизнес,сотрудник,crm,заказчик,менеджер,тестирование,тест,интерфейс
Topic # 02,name,function,var,user,data,end,return,self,type,запрос
Topic # 03,игра,игрок,игровой,играть,персонаж,unity,steam,движок,геймплей,жанр
Topic # 04,марс,земля,луна,космический,аппарат,планета,орбита,астероид,станция,солнечный
Topic # 05,файл,docker,скрипт,sudo,папка,file,пакет,http,git,linux
Topic # 06,android,google,ios,мобильный,app,устройство,layout,swift,view,play
Topic # 07,public,void,string,new,private,class,класс,static,return,null
Topic # 08,ракета,спутник,ступень,двигатель,spacex,космический,пуск,орбита,носитель,корабль
Topic # 09,css,javascript,angular,веб,react,vue,браузер,html,фронтенд,firefox
Topic # 10,атака,безопасность,уязвимость,устройство,злоумышленник,защита,пароль,вредоносный,сеть,доступ


In [34]:
topicsdf[2][1].T

,0,1,2,3,4,5,6,7,8,9
Topic # 01,сотрудник,продукт,бизнес,программист,деньга,зарплата,вакансия,специалист,менеджер,опыт
Topic # 02,function,var,return,self,user,data,name,url,def,new
Topic # 03,игра,игрок,игровой,играть,unity,персонаж,steam,геймплей,движок,жанр
Topic # 04,энергия,квантовый,частица,солнечный,вселенная,фотон,излучение,температура,хокинг,источник
Topic # 05,сеть,трафик,интернет,связь,оператор,dpi,сетевой,доступ,абонент,канал
Topic # 06,смартфон,телефон,аккумулятор,xiaomi,экран,дисплей,ноутбук,apple,батарея,iphone
Topic # 07,public,void,string,new,private,class,static,null,return,класс
Topic # 08,марс,луна,земля,космический,аппарат,планета,орбита,астероид,наса,станция
Topic # 09,css,javascript,angular,веб,react,vue,html,браузер,фронтенд,firefox
Topic # 10,атака,уязвимость,безопасность,злоумышленник,пароль,защита,вредоносный,ddos,ботнет,шифрование


Наиболее удачные темы получились при числе тем 30. Темы NMF модели более хорошо интерпретируемы, среди тематических слов меньше общих слов, чем в моделях LDA.
Темы лучшей модели NMF:

In [35]:
topicsdf[1][1].T

,0,1,2,3,4,5,6,7,8,9
Topic # 01,клиент,продукт,бизнес,сотрудник,crm,заказчик,менеджер,тестирование,тест,интерфейс
Topic # 02,name,function,var,user,data,end,return,self,type,запрос
Topic # 03,игра,игрок,игровой,играть,персонаж,unity,steam,движок,геймплей,жанр
Topic # 04,марс,земля,луна,космический,аппарат,планета,орбита,астероид,станция,солнечный
Topic # 05,файл,docker,скрипт,sudo,папка,file,пакет,http,git,linux
Topic # 06,android,google,ios,мобильный,app,устройство,layout,swift,view,play
Topic # 07,public,void,string,new,private,class,класс,static,return,null
Topic # 08,ракета,спутник,ступень,двигатель,spacex,космический,пуск,орбита,носитель,корабль
Topic # 09,css,javascript,angular,веб,react,vue,браузер,html,фронтенд,firefox
Topic # 10,атака,безопасность,уязвимость,устройство,злоумышленник,защита,пароль,вредоносный,сеть,доступ
